In [1]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
import nglview as nv

# https://biopython.org/docs/1.74/api/Bio.SVDSuperimposer.html
# conda install conda-forge::biopython
from Bio.SVDSuperimposer import SVDSuperimposer

from numpy import array, dot, set_printoptions

# # Path: pymdna/__init__.py, prototype of the package and atomic is not properly referenced in the package at genertors.py now I just explicitly define the path loction
# import pymdna as mdna
import sys
sys.path.append('/Users/thor/surfdrive/Projects/pymdna/')
import pymdna as mdna 
%load_ext autoreload
%autoreload 2

joblib is not installed. Falling back to sequential computation.


In [2]:
class SiteMapper:

    def __init__(self, s1s1, s2s2, segments=dict, k=100):
        
        self.segments = segments
        self.s1s1 = s1s1[::k]
        self.s2s2 = s2s2[::k]
        #self.get_site_map()
        
    def get_site_map(self):


        # get structures of the different sites
        s1 = self.get_site_structures(self.s1s1,site='s1')
        s2 = self.get_site_structures(self.s2s2,site='s2')

        h3_s1s1 = self.get_segment_structures(self.s1s1,site='h3')
        h3_s2s2 = self.get_segment_structures(self.s2s2,site='h3')
        h3 = md.join([h3_s1s1,h3_s2s2])

        l2_s1s1 = self.get_segment_structures(self.s1s1,site='l2')
        l2_s2s2 = self.get_segment_structures(self.s2s2,site='l2')
        l2 = md.join([l2_s1s1,l2_s2s2])

        dbd_s1s1 = self.get_segment_structures(self.s1s1,site='dbd')
        dbd_s2s2 = self.get_segment_structures(self.s2s2,site='dbd')
        dbd = md.join([dbd_s1s1,dbd_s2s2])

        # map of the different sites
        site_map = {'s1':s1,
                'h3':h3,
                's2':s2,
                'l2':l2,
                'dbd':dbd}
        
        return site_map

    def check_selection(self,top,selection):
        if selection == 'CA':
            indices = top.select('name CA')
        elif selection == 'backbone':
            indices = top.select('backbone')
        elif selection == 'sidechain':
            indices = top.select('sidechain')
        else:
            indices = top.select('all')   
        return indices 
    
    def get_monomer_domain_indices(self,top,domain,chain=0,selection=None):
        residues = np.array(top._chains[chain]._residues)
        indices = self.check_selection(top,selection)
        return [at.index for res in residues[domain] for at in res.atoms if at.index in indices]

    def get_segment_structures(self,traj,site='dbd'):
        chain_a = self.get_monomer_domain_indices(top=traj.top, domain=self.segments[site], chain=0, selection=None)
        chain_b = self.get_monomer_domain_indices(top=traj.top, domain=self.segments[site], chain=1, selection=None)
        A = traj.atom_slice(chain_a)
        B = traj.atom_slice(chain_b)
        return md.join([A,B])

    def get_site_structures(self, traj,site='s1'):
        chain_a = self.get_monomer_domain_indices(top=traj.top, domain=self.segments[site], chain=0, selection=None)
        chain_b = self.get_monomer_domain_indices(top=traj.top, domain=self.segments[site], chain=1, selection=None)
        return traj.atom_slice(np.sort(chain_a+chain_b))

    def show_domain(self, system, domains, domain):
        """"Not working yet, need to fix the selection of the atoms in the domain."""
        # shows first frame
        top = system.top
        view = nv.show_mdtraj(system[0])
        view.clear()
        indices = self.get_monomer_domain_indices(top, domains[domain], chain=0)
        view.add_representation('cartoon',selection=[i for i in  top.select('all') if i not in indices],color='cornflowerblue')
        top = system.topology
        chain_id = 0
        indices = self.get_monomer_domain_indices(top, domains[domain], chain=chain_id)
        view.add_representation('cartoon',selection=indices,color='gold')
        top = system.topology
        chain_id = 1
        indices = self.get_monomer_domain_indices(top, domains[domain], chain=chain_id)
        view.add_representation('cartoon',selection=indices,color='red')
        return view

In [6]:
# Load H-NS s1s1 dimers
loc_dimers = '/Users/thor/surfdrive/Projects/pymdna/studies/1_protein-dna_filament/FilamentsDNA/tests/data/0_s1s1/drytrajs/'
short_trajs = [md.load(loc_dimers+f'dry_{i}.xtc',top=loc_dimers+f'dry_{i}.pdb').remove_solvent() for i in range(0,2)]
#start_open = md.load(loc_dimers+f'dry_open.xtc',top=loc_dimers+f'dry_open.pdb').remove_solvent()
#start_closed = md.load(loc_dimers+f'dry_closed.xtc',top=loc_dimers+f'dry_closed.pdb').remove_solvent()
#s1s1 = md.join([start_open,start_closed,md.join(short_trajs)])
s1s1 = md.join(short_trajs)

# Load H-NS s2s2 dimers
loc_dimers = '/Users/thor/surfdrive/Projects/pymdna/studies/1_protein-dna_filament/FilamentsDNA/tests/data/1_s2s2/drytrajs/'
short_trajs = [md.load(loc_dimers+f'dry_{i}.xtc',top=loc_dimers+f'dry_{i}.pdb').remove_solvent() for i in range(0,2)]
s2s2 = md.join(short_trajs)

In [7]:
# Define segments of the protein
n = 2 # Overlap of residues between segments
segments = {'s1':np.arange(0,41+n),
            'h3':np.arange(41-n,53+n),
            's2':np.arange(53-n,82+n),
            'l2':np.arange(82-n,95+n),
            'dbd':np.arange(95-n,137)}

# Site map contains a dictionary with the trajectories of different sites of the protein catergorized from the s1s1 and s2s2 dimers
mapper = SiteMapper(s1s1, s2s2, segments=segments, k=100)
site_map = mapper.get_site_map()
site_map 

{'s1': <mdtraj.Trajectory with 26 frames, 1444 atoms, 86 residues, and unitcells at 0x189a3c350>,
 'h3': <mdtraj.Trajectory with 134 frames, 243 atoms, 16 residues, and unitcells at 0x189bce550>,
 's2': <mdtraj.Trajectory with 41 frames, 1072 atoms, 66 residues, and unitcells at 0x189aeef50>,
 'l2': <mdtraj.Trajectory with 134 frames, 276 atoms, 17 residues, and unitcells at 0x189d08cd0>,
 'dbd': <mdtraj.Trajectory with 134 frames, 690 atoms, 44 residues, and unitcells at 0x189fa44d0>}

In [ ]:
##### Functions to build protein filaments using segements and rotation, transation transformation fits #####

def get_rot_and_trans(subtraj_A,subtraj_B):
    
    """ fit only works now on a single frame (mdtraj returns xyz with shape (n_frames, atoms, xyz) 
         even for single frame trajs so hence the xyz[0]"""
    
    # load super imposer
    sup = SVDSuperimposer()

    # Set the coords, y will be rotated and translated on x
    x = subtraj_A.xyz[0]
    y = subtraj_B.xyz[0]
    sup.set(x, y)

    # Do the leastsquared fit
    sup.run()

    # Get the rms
    rms = sup.get_rms()

    # Get rotation (right multiplying!) and the translation
    rot, tran = sup.get_rotran()
    
    # now we have the instructions to rotate B on A
    return rot,tran,rms

def apply_superimposition(traj, rot, tran):
    
    # get xyz coordinates
    xyz = traj.xyz[0]
    
    # rotate subject on target
    new_xyz = dot(xyz, rot) + tran

    # replace coordinates of traj
    traj.xyz = new_xyz
    return traj

def fit_B_on_A(A, B, overlap_A, overlap_B):
    # create trajs containing only the selections
    subtraj_A = A.atom_slice(overlap_A)
    subtraj_B = B.atom_slice(overlap_B)

    # obtain instructions to rotate and translate B on A based on substraj structures
    rot, tran, rms = get_rot_and_trans(subtraj_A,subtraj_B)

    # do the superimposition of B on A and subsitute old with new xyz of B
    return apply_superimposition(B, rot, tran)
    

##### Functions to edit and mange topology and trajectory #####

def get_overlap_indices(top,n,chain=0,terminus=None):
    residues = np.array(top._chains[chain]._residues)
    if terminus == 'N_terminus': # get residues at end of chain
        s = residues[len(residues)-n*2:len(residues)]
        return [at.index for res in s for at in res.atoms]
    elif terminus == 'C_terminus': # get residues at beginning of chain
        s = residues[:n*2]
        return [at.index for res in s for at in res.atoms]
    else:
        print('No terminus')
        
def check_if_dimerization(site):
    if 's' in site:
        return True
    else:
        return False
    
def get_termini(site_x,site_y):
    chain_order = np.array(['s1','h3','s2','l2','dbd'])
    x = np.argwhere(chain_order==site_x)
    y = np.argwhere(chain_order==site_y)
    if x < y:
        return ['N_terminus','C_terminus']
    elif x > y:
        return ['C_terminus','N_terminus']

def check_overlaps(overlap_A,overlap_B):

    if len(overlap_A) != len(overlap_B):
        print(len(overlap_A),len(overlap_B))
        print('Something went wrong with finding the overlaps') 
    else:
        False

def remove_overlap(traj,overlap):
     return traj.atom_slice([at.index for at in traj.top.atoms if at.index not in overlap])
    
def split_chain_topology(traj,leading_chain):
    # split part of A in chain that is being extended and that is not
    traj_active = traj.atom_slice(traj.top.select(f'chainid {leading_chain}'))
    traj_passive = traj.atom_slice(traj.top.select(f'not chainid {leading_chain}'))
    return traj_active, traj_passive

def merge_chain_topology(A,B,keep_resSeq=True):
    C = A.stack(B,keep_resSeq=keep_resSeq)
    top = C.top
    # Merge two tops (with two chains or more) to a top of one chain 
    out = md.Topology()
    c = out.add_chain()
    for chain in top.chains:

        for residue in chain.residues:
            r = out.add_residue(residue.name, c, residue.resSeq, residue.segment_id)
            for atom in residue.atoms:
                out.add_atom(atom.name, atom.element, r, serial=atom.serial)
    #     for bond in top.bonds:
    #         a1, a2 = bond
    #         out.add_bond(a1, a2, type=bond.type, order=bond.order)
    out.create_standard_bonds() #rare manier om bonds te maken, maar werkt
    C.top = out 
    return C

##### Functions to orchestrate the addition of protein segments to form dimers ##### 

def add_pair(traj,pair,site_map,leading_chain=0,adding_chain=0,verbose=False,reverse=False,segment='fixed'):

    keep_resSeq = False
    A,B,C=None,None,None
    site_a, site_b = pair
    if verbose:
        print(site_a,site_b)
    
    # get segment structures
    if not traj:
        if segment == 'fixed':
            x,y = 40,90 # get fixed frame for segment
        elif segment == 'random':
            k = len(site_map[site_a])
            l = len(site_map[site_b])
            x,y = np.random.randint(0,k,1)[0],np.random.randint(0,l,1)[0]
        A = site_map[site_a][x]
        B = site_map[site_b][y]
    else:
        if segment == 'fixed':
            z = 20
        elif segment == 'random':
            k = len(site_map[site_b])
            z = np.random.randint(0,k,1)[0]
        A = traj
        B = site_map[site_b][z]
        
    # check if site had dimerization site
    dimer_a = check_if_dimerization(site_a)
    dimer_b = check_if_dimerization(site_b)
    if verbose:
        print(dimer_a, dimer_b)
        
    # get_termini of site a and b
    terminus_a, terminus_b = get_termini(site_a,site_b)
    
    # determine growth direction (forward, or reverse)
    if terminus_a == 'C_terminus':
        reverse = True
    else:
        reverse = False
        
    # get atom indices of overlapping segements
    overlap_A = get_overlap_indices(A.top,n,chain=leading_chain,terminus=terminus_a)
    overlap_B = get_overlap_indices(B.top,n,chain=adding_chain,terminus=terminus_b)
    
    # make sure overlapping indices are consistent
    check = check_overlaps(overlap_A,overlap_B)
    if check:
        return check
    
    # obtain superimposition of B on A
    new_B = fit_B_on_A(A,B,overlap_A,overlap_B)
    
    # remove overlapping selection from A used for fit
    new_A = remove_overlap(A,overlap_A)
    
    # splits topology in leading chain and remainder (not leading chain(s))
    A_active, A_passive = split_chain_topology(new_A,leading_chain)
    
    if dimer_b:
        # splits topology in leading chain and remainder (not adding chain(s))
        B_active, B_passive = split_chain_topology(new_B,adding_chain)
            
        # add B to active part of A (and make sure they are in same chain)
        if reverse:
            temp = merge_chain_topology(B_active,A_active,keep_resSeq=keep_resSeq)
        else:
            temp = merge_chain_topology(A_active,B_active,keep_resSeq=keep_resSeq)
            
        C_temp = temp.stack(A_passive,keep_resSeq=keep_resSeq)
        C =  C_temp.stack(B_passive,keep_resSeq=keep_resSeq)
    else:
        # add B to active part of A (and make sure they are in same chain)
        if reverse:
            temp = merge_chain_topology(new_B,A_active)
        else:
            temp = merge_chain_topology(A_active,new_B)
        # combine passive part with new structure (active part of A and B)
        C = temp.stack(A_passive,keep_resSeq=keep_resSeq)
        
    return C

def add_dimer(traj, chainid = 0,  site_map=None, verbose=False, segment='random'):
    
    s1_pairs = [['s1', 'h3'],['h3','s2'],['s2','l2'],['l2','dbd']]
    s2_pairs = [['s2','h3'],['h3','s1'],['s2', 'l2'],['l2','dbd']]
    

    for idx,pair in enumerate(s2_pairs):
        if idx > 0:
            leading_chain = 0
        else:
            leading_chain = chainid
        traj = add_pair(traj,pair,site_map,leading_chain=leading_chain,verbose=verbose,segment=segment)

    for idx,pair in enumerate(s1_pairs):
        if idx > 0:
            leading_chain = 0
        else:
            leading_chain =  chainid + 2
        traj = add_pair(traj,pair,site_map,leading_chain=leading_chain,verbose=verbose,segment=segment)
    return traj


# initialize number of dimers
traj = None
dimers = 2

i = 0
for idx in range(dimers):
    print(idx)
    traj = add_dimer(traj, chainid=i, site_map=site_map, segment='random')
    i+=2

# show first frame
print([c.n_residues for c in traj.top.chains])


view = nv.show_mdtraj(traj.atom_slice(traj.top.select(f'chainid 0 to {(dimers*2)-1}')))
view

In [20]:
##### Functions to build protein filaments using segements and rotation, transation transformation fits #####

def get_rot_and_trans(subtraj_A,subtraj_B):
    
    """ fit only works now on a single frame (mdtraj returns xyz with shape (n_frames, atoms, xyz) 
         even for single frame trajs so hence the xyz[0]"""
    
    # load super imposer
    sup = SVDSuperimposer()

    # Set the coords, y will be rotated and translated on x
    x = subtraj_A.xyz[0]
    y = subtraj_B.xyz[0]
    sup.set(x, y)

    # Do the leastsquared fit
    sup.run()

    # Get the rms
    rms = sup.get_rms()

    # Get rotation (right multiplying!) and the translation
    rot, tran = sup.get_rotran()
    
    # now we have the instructions to rotate B on A
    return rot,tran,rms

def apply_superimposition(traj, rot, tran):
    
    # get xyz coordinates
    xyz = traj.xyz[0]
    
    # rotate subject on target
    new_xyz = dot(xyz, rot) + tran

    # replace coordinates of traj
    traj.xyz = new_xyz
    return traj

def fit_B_on_A(A, B, overlap_A, overlap_B):
    # create trajs containing only the selections
    subtraj_A = A.atom_slice(overlap_A)
    subtraj_B = B.atom_slice(overlap_B)

    # obtain instructions to rotate and translate B on A based on substraj structures
    rot, tran, rms = get_rot_and_trans(subtraj_A,subtraj_B)

    # do the superimposition of B on A and subsitute old with new xyz of B
    return apply_superimposition(B, rot, tran)


##### Functions to edit and mange topology and trajectory #####

def check_if_dimerization(site):
    if 's' in site:
        return True
    else:
        return False

def get_termini(site_x,site_y):
    chain_order = np.array(['s1','h3','s2','l2','dbd'])
    x = np.argwhere(chain_order==site_x)
    y = np.argwhere(chain_order==site_y)
    if x < y:
        return ['N_terminus','C_terminus']
    elif x > y:
        return ['C_terminus','N_terminus']

def get_overlap_indices(top,n=2,chain=0,terminus=None):
    residues = np.array(top._chains[chain]._residues)
    if terminus == 'N_terminus': # get residues at end of chain
        s = residues[len(residues)-n*2:len(residues)]
        return [at.index for res in s for at in res.atoms]
    elif terminus == 'C_terminus': # get residues at beginning of chain
        s = residues[:n*2]
        return [at.index for res in s for at in res.atoms]
    else:
        print('No terminus')

def check_overlaps(overlap_A,overlap_B):

    if len(overlap_A) != len(overlap_B):
        print(len(overlap_A),len(overlap_B))
        print('Something went wrong with finding the overlaps') 
    else:
        False

def remove_overlap(traj,overlap):
     return traj.atom_slice([at.index for at in traj.top.atoms if at.index not in overlap])
    
def split_chain_topology(traj,leading_chain):
    # split part of A in chain that is being extended and that is not
    traj_active = traj.atom_slice(traj.top.select(f'chainid {leading_chain}'))
    traj_passive = traj.atom_slice(traj.top.select(f'not chainid {leading_chain}'))
    return traj_active, traj_passive

def merge_chain_topology(A,B,keep_resSeq=True):
    C = A.stack(B,keep_resSeq=keep_resSeq)
    top = C.top
    # Merge two tops (with two chains or more) to a top of one chain 
    out = md.Topology()
    c = out.add_chain()
    for chain in top.chains:

        for residue in chain.residues:
            r = out.add_residue(residue.name, c, residue.resSeq, residue.segment_id)
            for atom in residue.atoms:
                out.add_atom(atom.name, atom.element, r, serial=atom.serial)
    #     for bond in top.bonds:
    #         a1, a2 = bond
    #         out.add_bond(a1, a2, type=bond.type, order=bond.order)
    out.create_standard_bonds() #rare manier om bonds te maken, maar werkt
    C.top = out 
    return C

    
def select_segment_frame(segment, site_a, site_b, site_map):
    """
    Selects frame indices based on the segment type.
    
    Args:
        segment (str): Type of segment, either 'fixed' or 'random'.
        site_a (str): Identifier for site A.
        site_b (str): Identifier for site B.
        site_map (dict): Mapping of sites to their respective frames.
        
    Returns:
        tuple: Selected frame indices for site A and site B.
    """
    if segment == 'fixed':
        return 40, 90  # Example fixed indices, replace with actual logic if available
    elif segment == 'random':
        return np.random.randint(0, len(site_map[site_a]), 1)[0], np.random.randint(0, len(site_map[site_b]), 1)[0]

def get_sites(site_a, site_b, traj, site_map, segment):
    """
    Retrieves the site structures based on whether a trajectory is provided.
    
    Args:
        site_a (str): Identifier for site A.
        site_b (str): Identifier for site B.
        traj (object): Trajectory object, None if not available.
        site_map (dict): Mapping of sites to their frames.
        segment (str): Segment type.
        
    Returns:
        tuple: Structures for site A and site B.
    """
    if not traj:
        x, y = select_segment_frame(segment, site_a, site_b, site_map)
        return site_map[site_a][x], site_map[site_b][y]
    else:
        z = 20 if segment == 'fixed' else np.random.randint(0, len(site_map[site_b]), 1)[0]
        return traj, site_map[site_b][z]

def handle_reversal(terminus_a):
    """
    Determines whether to reverse the growth direction based on the terminus type.
    
    Args:
        terminus_a (str): Type of terminus at site A ('C_terminus' or other).
    
    Returns:
        bool: True if reverse growth direction, False otherwise.
    """
    return terminus_a == 'C_terminus'

def add_chain_topology(A, B, reverse, leading_chain, adding_chain, keep_resSeq):
    """
    Handles the topology merging between two chain segments.
    
    Args:
        A (object): Structure of the leading chain.
        B (object): Structure of the adding chain.
        reverse (bool): Whether to reverse the order of merging.
        leading_chain (int): Index of the leading chain.
        adding_chain (int): Index of the adding chain.
        keep_resSeq (bool): Whether to keep the original residue sequence numbers.
    
    Returns:
        object: New combined structure.
    """
    A_active, A_passive = split_chain_topology(A, leading_chain)
    B_active, B_passive = split_chain_topology(B, adding_chain)
    temp = merge_chain_topology(B_active if reverse else A_active, A_active if reverse else B_active, keep_resSeq)
    C_temp = temp.stack(A_passive, keep_resSeq=keep_resSeq)
    return C_temp.stack(B_passive, keep_resSeq=keep_resSeq)

def add_pair(traj, pair, site_map, leading_chain=0, adding_chain=0, n_overlap=2, verbose=False, reverse=False, segment='fixed'):
    """
    Constructs a new molecular structure by adding two chain segments.
    
    Args:
        traj (object): Molecular trajectory, None if not present.
        pair (tuple): Pair of site identifiers (site_a, site_b).
        site_map (dict): Mapping of sites to frames.
        leading_chain (int): Index of the leading chain in the topology.
        adding_chain (int): Index of the chain being added.
        verbose (bool): If True, prints debug information.
        reverse (bool): If True, reverses the merge order.
        segment (str): Type of segment selection.
    
    Returns:
        object: New molecular structure after adding the chains.
    """
    keep_resSeq = False
    site_a, site_b = pair

    if verbose:
        print(f"Processing pair: {site_a}, {site_b}")

    A, B = get_sites(site_a, site_b, traj, site_map, segment)
    dimer_a = check_if_dimerization(site_a)
    dimer_b = check_if_dimerization(site_b)
    terminus_a, terminus_b = get_termini(site_a, site_b)

    reverse = handle_reversal(terminus_a)

    overlap_A = get_overlap_indices(A.top, n=n_overlap, chain=leading_chain, terminus=terminus_a)
    overlap_B = get_overlap_indices(B.top, n=n_overlap, chain=adding_chain, terminus=terminus_b)

    if check_overlaps(overlap_A, overlap_B):
        return "Overlap check failed."

    new_B = fit_B_on_A(A, B, overlap_A, overlap_B)
    new_A = remove_overlap(A, overlap_A)

    if dimer_b:
        C = add_chain_topology(new_A, new_B, reverse, leading_chain, adding_chain, keep_resSeq)
    else:
        temp = merge_chain_topology(new_B if reverse else new_A, new_A if reverse else new_B)
        C = temp.stack(new_A, keep_resSeq=keep_resSeq)

    if verbose:
        print("Construction completed successfully.")

    return C

def add_dimer(traj, chainid = 0,  site_map=None, verbose=False, segment='random', n_overlap=2):
    
    s1_pairs = [['s1', 'h3'],['h3','s2'],['s2','l2'],['l2','dbd']]
    s2_pairs = [['s2','h3'],['h3','s1'],['s2', 'l2'],['l2','dbd']]
    

    for idx,pair in enumerate(s2_pairs):
        if idx > 0:
            leading_chain = 0
        else:
            leading_chain = chainid
        traj = add_pair(traj,pair,site_map,leading_chain=leading_chain,n_overlap=n_overlap,verbose=verbose,segment=segment)

    for idx,pair in enumerate(s1_pairs):
        if idx > 0:
            leading_chain = 0
        else:
            leading_chain =  chainid + 2
        traj = add_pair(traj,pair,site_map,leading_chain=leading_chain,n_overlap=n_overlap,verbose=verbose,segment=segment)
    return traj


    # initialize number of dimers
traj = None
dimers = 2

i = 0
for idx in range(dimers):
    print(idx)
    traj = add_dimer(traj, chainid=i, site_map=site_map, segment='random', n_overlap=n)
    i+=2

# show first frame
print([c.n_residues for c in traj.top.chains])

0
54 78
Something went wrong with finding the overlaps


Exception: Coordinate number/dimension mismatch.

In [14]:
##### Functions to build protein filaments using segements and rotation, transation transformation fits #####

def get_rot_and_trans(subtraj_A,subtraj_B):
    
    """ fit only works now on a single frame (mdtraj returns xyz with shape (n_frames, atoms, xyz) 
         even for single frame trajs so hence the xyz[0]"""
    
    # load super imposer
    sup = SVDSuperimposer()

    # Set the coords, y will be rotated and translated on x
    x = subtraj_A.xyz[0]
    y = subtraj_B.xyz[0]
    sup.set(x, y)

    # Do the leastsquared fit
    sup.run()

    # Get the rms
    rms = sup.get_rms()

    # Get rotation (right multiplying!) and the translation
    rot, tran = sup.get_rotran()
    
    # now we have the instructions to rotate B on A
    return rot,tran,rms

def apply_superimposition(traj, rot, tran):
    
    # get xyz coordinates
    xyz = traj.xyz[0]
    
    # rotate subject on target
    new_xyz = dot(xyz, rot) + tran

    # replace coordinates of traj
    traj.xyz = new_xyz
    return traj

def fit_B_on_A(A, B, overlap_A, overlap_B):
    # create trajs containing only the selections
    subtraj_A = A.atom_slice(overlap_A)
    subtraj_B = B.atom_slice(overlap_B)

    # obtain instructions to rotate and translate B on A based on substraj structures
    rot, tran, rms = get_rot_and_trans(subtraj_A,subtraj_B)

    # do the superimposition of B on A and subsitute old with new xyz of B
    return apply_superimposition(B, rot, tran)
    

##### Functions to edit and mange topology and trajectory #####

def get_overlap_indices(top,n,chain=0,terminus=None):
    residues = np.array(top._chains[chain]._residues)
    if terminus == 'N_terminus': # get residues at end of chain
        s = residues[len(residues)-n*2:len(residues)]
        return [at.index for res in s for at in res.atoms]
    elif terminus == 'C_terminus': # get residues at beginning of chain
        s = residues[:n*2]
        return [at.index for res in s for at in res.atoms]
    else:
        print('No terminus')
        
def check_if_dimerization(site):
    if 's' in site:
        return True
    else:
        return False
    
def get_termini(site_x,site_y):
    chain_order = np.array(['s1','h3','s2','l2','dbd'])
    x = np.argwhere(chain_order==site_x)
    y = np.argwhere(chain_order==site_y)
    if x < y:
        return ['N_terminus','C_terminus']
    elif x > y:
        return ['C_terminus','N_terminus']

def check_overlaps(overlap_A,overlap_B):

    if len(overlap_A) != len(overlap_B):
        print(len(overlap_A),len(overlap_B))
        print('Something went wrong with finding the overlaps') 
    else:
        False

def remove_overlap(traj,overlap):
     return traj.atom_slice([at.index for at in traj.top.atoms if at.index not in overlap])
    
def split_chain_topology(traj,leading_chain):
    # split part of A in chain that is being extended and that is not
    traj_active = traj.atom_slice(traj.top.select(f'chainid {leading_chain}'))
    traj_passive = traj.atom_slice(traj.top.select(f'not chainid {leading_chain}'))
    return traj_active, traj_passive

def merge_chain_topology(A,B,keep_resSeq=True):
    C = A.stack(B,keep_resSeq=keep_resSeq)
    top = C.top
    # Merge two tops (with two chains or more) to a top of one chain 
    out = md.Topology()
    c = out.add_chain()
    for chain in top.chains:

        for residue in chain.residues:
            r = out.add_residue(residue.name, c, residue.resSeq, residue.segment_id)
            for atom in residue.atoms:
                out.add_atom(atom.name, atom.element, r, serial=atom.serial)
    #     for bond in top.bonds:
    #         a1, a2 = bond
    #         out.add_bond(a1, a2, type=bond.type, order=bond.order)
    out.create_standard_bonds() #rare manier om bonds te maken, maar werkt
    C.top = out 
    return C


In [16]:
import numpy as np

def select_segment_frame(segment, site_a, site_b, site_map):
    """
    Selects frame indices based on the segment type.
    
    Args:
        segment (str): Type of segment, either 'fixed' or 'random'.
        site_a (str): Identifier for site A.
        site_b (str): Identifier for site B.
        site_map (dict): Mapping of sites to their respective frames.
        
    Returns:
        tuple: Selected frame indices for site A and site B.
    """
    if segment == 'fixed':
        return 40, 90  # Example fixed indices, replace with actual logic if available
    elif segment == 'random':
        return np.random.randint(0, len(site_map[site_a]), 1)[0], np.random.randint(0, len(site_map[site_b]), 1)[0]

def get_sites(site_a, site_b, traj, site_map, segment):
    """
    Retrieves the site structures based on whether a trajectory is provided.
    
    Args:
        site_a (str): Identifier for site A.
        site_b (str): Identifier for site B.
        traj (object): Trajectory object, None if not available.
        site_map (dict): Mapping of sites to their frames.
        segment (str): Segment type.
        
    Returns:
        tuple: Structures for site A and site B.
    """
    if not traj:
        x, y = select_segment_frame(segment, site_a, site_b, site_map)
        return site_map[site_a][x], site_map[site_b][y]
    else:
        z = 20 if segment == 'fixed' else np.random.randint(0, len(site_map[site_b]), 1)[0]
        return traj, site_map[site_b][z]

def handle_reversal(terminus_a):
    """
    Determines whether to reverse the growth direction based on the terminus type.
    
    Args:
        terminus_a (str): Type of terminus at site A ('C_terminus' or other).
    
    Returns:
        bool: True if reverse growth direction, False otherwise.
    """
    return terminus_a == 'C_terminus'

def add_chain_topology(A, B, reverse, leading_chain, adding_chain, keep_resSeq):
    """
    Handles the topology merging between two chain segments.
    
    Args:
        A (object): Structure of the leading chain.
        B (object): Structure of the adding chain.
        reverse (bool): Whether to reverse the order of merging.
        leading_chain (int): Index of the leading chain.
        adding_chain (int): Index of the adding chain.
        keep_resSeq (bool): Whether to keep the original residue sequence numbers.
    
    Returns:
        object: New combined structure.
    """
    A_active, A_passive = split_chain_topology(A, leading_chain)
    B_active, B_passive = split_chain_topology(B, adding_chain)
    temp = merge_chain_topology(B_active if reverse else A_active, A_active if reverse else B_active, keep_resSeq)
    C_temp = temp.stack(A_passive, keep_resSeq=keep_resSeq)
    return C_temp.stack(B_passive, keep_resSeq=keep_resSeq)

def add_pair(traj, pair, site_map, leading_chain=0, adding_chain=0, verbose=False, reverse=False, segment='fixed'):
    """
    Constructs a new molecular structure by adding two chain segments.
    
    Args:
        traj (object): Molecular trajectory, None if not present.
        pair (tuple): Pair of site identifiers (site_a, site_b).
        site_map (dict): Mapping of sites to frames.
        leading_chain (int): Index of the leading chain in the topology.
        adding_chain (int): Index of the chain being added.
        verbose (bool): If True, prints debug information.
        reverse (bool): If True, reverses the merge order.
        segment (str): Type of segment selection.
    
    Returns:
        object: New molecular structure after adding the chains.
    """
    keep_resSeq = False
    site_a, site_b = pair

    if verbose:
        print(f"Processing pair: {site_a}, {site_b}")

    A, B = get_sites(site_a, site_b, traj, site_map, segment)
    dimer_a = check_if_dimerization(site_a)
    dimer_b = check_if_dimerization(site_b)
    terminus_a, terminus_b = get_termini(site_a, site_b)

    reverse = handle_reversal(terminus_a)

    overlap_A = get_overlap_indices(A.top, chain=leading_chain, terminus=terminus_a)
    overlap_B = get_overlap_indices(B.top, chain=adding_chain, terminus=terminus_b)

    if check_overlaps(overlap_A, overlap_B):
        return "Overlap check failed."

    new_B = fit_B_on_A(A, B, overlap_A, overlap_B)
    new_A = remove_overlap(A, overlap_A)

    if dimer_b:
        C = add_chain_topology(new_A, new_B, reverse, leading_chain, adding_chain, keep_resSeq)
    else:
        temp = merge_chain_topology(new_B if reverse else new_A, new_A if reverse else new_B)
        C = temp.stack(new_A, keep_resSeq=keep_resSeq)

    if verbose:
        print("Construction completed successfully.")

    return C


def add_dimer(traj, chainid = 0,  site_map=None, verbose=False, segment='random'):
    
    s1_pairs = [['s1', 'h3'],['h3','s2'],['s2','l2'],['l2','dbd']]
    s2_pairs = [['s2','h3'],['h3','s1'],['s2', 'l2'],['l2','dbd']]
    

    for idx,pair in enumerate(s2_pairs):
        if idx > 0:
            leading_chain = 0
        else:
            leading_chain = chainid
        traj = add_pair(traj,pair,site_map,leading_chain=leading_chain,verbose=verbose,segment=segment)

    for idx,pair in enumerate(s1_pairs):
        if idx > 0:
            leading_chain = 0
        else:
            leading_chain =  chainid + 2
        traj = add_pair(traj,pair,site_map,leading_chain=leading_chain,verbose=verbose,segment=segment)
    return traj

In [ ]:
import numpy as np

def select_segment_frame(segment, site_a, site_b, site_map):
    """
    Selects frame indices based on the segment type.
    
    Args:
        segment (str): Type of segment, either 'fixed' or 'random'.
        site_a (str): Identifier for site A.
        site_b (str): Identifier for site B.
        site_map (dict): Mapping of sites to their respective frames.
        
    Returns:
        tuple: Selected frame indices for site A and site B.
    """
    if segment == 'fixed':
        return 40, 90  # Example fixed indices, replace with actual logic if available
    elif segment == 'random':
        return np.random.randint(0, len(site_map[site_a]), 1)[0], np.random.randint(0, len(site_map[site_b]), 1)[0]

def get_sites(site_a, site_b, traj, site_map, segment):
    """
    Retrieves the site structures based on whether a trajectory is provided.
    
    Args:
        site_a (str): Identifier for site A.
        site_b (str): Identifier for site B.
        traj (object): Trajectory object, None if not available.
        site_map (dict): Mapping of sites to their frames.
        segment (str): Segment type.
        
    Returns:
        tuple: Structures for site A and site B.
    """
    if not traj:
        x, y = select_segment_frame(segment, site_a, site_b, site_map)
        return site_map[site_a][x], site_map[site_b][y]
    else:
        z = 20 if segment == 'fixed' else np.random.randint(0, len(site_map[site_b]), 1)[0]
        return traj, site_map[site_b][z]

def handle_reversal(terminus_a):
    """
    Determines whether to reverse the growth direction based on the terminus type.
    
    Args:
        terminus_a (str): Type of terminus at site A ('C_terminus' or other).
    
    Returns:
        bool: True if reverse growth direction, False otherwise.
    """
    return terminus_a == 'C_terminus'

def add_chain_topology(A, B, reverse, leading_chain, adding_chain, keep_resSeq):
    """
    Handles the topology merging between two chain segments.
    
    Args:
        A (object): Structure of the leading chain.
        B (object): Structure of the adding chain.
        reverse (bool): Whether to reverse the order of merging.
        leading_chain (int): Index of the leading chain.
        adding_chain (int): Index of the adding chain.
        keep_resSeq (bool): Whether to keep the original residue sequence numbers.
    
    Returns:
        object: New combined structure.
    """
    A_active, A_passive = split_chain_topology(A, leading_chain)
    B_active, B_passive = split_chain_topology(B, adding_chain)
    temp = merge_chain_topology(B_active if reverse else A_active, A_active if reverse else B_active, keep_resSeq)
    C_temp = temp.stack(A_passive, keep_resSeq=keep_resSeq)
    return C_temp.stack(B_passive, keep_resSeq=keep_resSeq)

def add_pair(traj, pair, site_map, leading_chain=0, adding_chain=0, verbose=False, reverse=False, segment='fixed'):
    """
    Constructs a new molecular structure by adding two chain segments.
    
    Args:
        traj (object): Molecular trajectory, None if not present.
        pair (tuple): Pair of site identifiers (site_a, site_b).
        site_map (dict): Mapping of sites to frames.
        leading_chain (int): Index of the leading chain in the topology.
        adding_chain (int): Index of the chain being added.
        verbose (bool): If True, prints debug information.
        reverse (bool): If True, reverses the merge order.
        segment (str): Type of segment selection.
    
    Returns:
        object: New molecular structure after adding the chains.
    """
    keep_resSeq = False
    site_a, site_b = pair

    if verbose:
        print(f"Processing pair: {site_a}, {site_b}")

    A, B = get_sites(site_a, site_b, traj, site_map, segment)
    dimer_a = check_if_dimerization(site_a)
    dimer_b = check_if_dimerization(site_b)
    terminus_a, terminus_b = get_termini(site_a, site_b)

    reverse = handle_reversal(terminus_a)

    overlap_A = get_overlap_indices(A.top, chain=leading_chain, terminus=terminus_a)
    overlap_B = get_overlap_indices(B.top, chain=adding_chain, terminus=terminus_b)

    if check_overlaps(overlap_A, overlap_B):
        return "Overlap check failed."

    new_B = fit_B_on_A(A, B, overlap_A, overlap_B)
    new_A = remove_overlap(A, overlap_A)

    if dimer_b:
        C = add_chain_topology(new_A, new_B, reverse, leading_chain, adding_chain, keep_resSeq)
    else:
        temp = merge_chain_topology(new_B if reverse else new_A, new_A if reverse else new_B)
        C = temp.stack(new_A, keep_resSeq=keep_resSeq)

    if verbose:
        print("Construction completed successfully.")

    return C

In [21]:
##### Functions to build protein filaments using segements and rotation, transation transformation fits #####

def get_rot_and_trans(subtraj_A,subtraj_B):
    
    """ fit only works now on a single frame (mdtraj returns xyz with shape (n_frames, atoms, xyz) 
         even for single frame trajs so hence the xyz[0]"""
    
    # load super imposer
    sup = SVDSuperimposer()

    # Set the coords, y will be rotated and translated on x
    x = subtraj_A.xyz[0]
    y = subtraj_B.xyz[0]
    sup.set(x, y)

    # Do the leastsquared fit
    sup.run()

    # Get the rms
    rms = sup.get_rms()

    # Get rotation (right multiplying!) and the translation
    rot, tran = sup.get_rotran()
    
    # now we have the instructions to rotate B on A
    return rot,tran,rms

def apply_superimposition(traj, rot, tran):
    
    # get xyz coordinates
    xyz = traj.xyz[0]
    
    # rotate subject on target
    new_xyz = dot(xyz, rot) + tran

    # replace coordinates of traj
    traj.xyz = new_xyz
    return traj

def fit_B_on_A(A, B, overlap_A, overlap_B):
    # create trajs containing only the selections
    subtraj_A = A.atom_slice(overlap_A)
    subtraj_B = B.atom_slice(overlap_B)

    # obtain instructions to rotate and translate B on A based on substraj structures
    rot, tran, rms = get_rot_and_trans(subtraj_A,subtraj_B)

    # do the superimposition of B on A and subsitute old with new xyz of B
    return apply_superimposition(B, rot, tran)
    

##### Functions to edit and mange topology and trajectory #####

def get_overlap_indices(top,n,chain=0,terminus=None):
    residues = np.array(top._chains[chain]._residues)
    if terminus == 'N_terminus': # get residues at end of chain
        s = residues[len(residues)-n*2:len(residues)]
        return [at.index for res in s for at in res.atoms]
    elif terminus == 'C_terminus': # get residues at beginning of chain
        s = residues[:n*2]
        return [at.index for res in s for at in res.atoms]
    else:
        print('No terminus')
        
def check_if_dimerization(site):
    if 's' in site:
        return True
    else:
        return False
    
def get_termini(site_x,site_y):
    chain_order = np.array(['s1','h3','s2','l2','dbd'])
    x = np.argwhere(chain_order==site_x)
    y = np.argwhere(chain_order==site_y)
    if x < y:
        return ['N_terminus','C_terminus']
    elif x > y:
        return ['C_terminus','N_terminus']

def check_overlaps(overlap_A,overlap_B):

    if len(overlap_A) != len(overlap_B):
        print(len(overlap_A),len(overlap_B))
        print('Something went wrong with finding the overlaps') 
    else:
        False

def remove_overlap(traj,overlap):
     return traj.atom_slice([at.index for at in traj.top.atoms if at.index not in overlap])
    
def split_chain_topology(traj,leading_chain):
    # split part of A in chain that is being extended and that is not
    traj_active = traj.atom_slice(traj.top.select(f'chainid {leading_chain}'))
    traj_passive = traj.atom_slice(traj.top.select(f'not chainid {leading_chain}'))
    return traj_active, traj_passive

def merge_chain_topology(A,B,keep_resSeq=True):
    C = A.stack(B,keep_resSeq=keep_resSeq)
    top = C.top
    # Merge two tops (with two chains or more) to a top of one chain 
    out = md.Topology()
    c = out.add_chain()
    for chain in top.chains:

        for residue in chain.residues:
            r = out.add_residue(residue.name, c, residue.resSeq, residue.segment_id)
            for atom in residue.atoms:
                out.add_atom(atom.name, atom.element, r, serial=atom.serial)
    #     for bond in top.bonds:
    #         a1, a2 = bond
    #         out.add_bond(a1, a2, type=bond.type, order=bond.order)
    out.create_standard_bonds() #rare manier om bonds te maken, maar werkt
    C.top = out 
    return C

##### Functions to orchestrate the addition of protein segments to form dimers ##### 

def add_pair(traj,pair,site_map,leading_chain=0,adding_chain=0,verbose=False,reverse=False,segment='fixed'):

    keep_resSeq = False
    A,B,C=None,None,None
    site_a, site_b = pair
    if verbose:
        print(site_a,site_b)
    
    # get segment structures
    if not traj:
        if segment == 'fixed':
            x,y = 40,90 # get fixed frame for segment
        elif segment == 'random':
            k = len(site_map[site_a])
            l = len(site_map[site_b])
            x,y = np.random.randint(0,k,1)[0],np.random.randint(0,l,1)[0]
        A = site_map[site_a][x]
        B = site_map[site_b][y]
    else:
        if segment == 'fixed':
            z = 20
        elif segment == 'random':
            k = len(site_map[site_b])
            z = np.random.randint(0,k,1)[0]
        A = traj
        B = site_map[site_b][z]
        
    # check if site had dimerization site
    dimer_a = check_if_dimerization(site_a)
    dimer_b = check_if_dimerization(site_b)
    if verbose:
        print(dimer_a, dimer_b)
        
    # get_termini of site a and b
    terminus_a, terminus_b = get_termini(site_a,site_b)
    
    # determine growth direction (forward, or reverse)
    if terminus_a == 'C_terminus':
        reverse = True
    else:
        reverse = False
        
    # get atom indices of overlapping segements
    overlap_A = get_overlap_indices(A.top,n,chain=leading_chain,terminus=terminus_a)
    overlap_B = get_overlap_indices(B.top,n,chain=adding_chain,terminus=terminus_b)
    
    # make sure overlapping indices are consistent
    check = check_overlaps(overlap_A,overlap_B)
    if check:
        return check
    
    # obtain superimposition of B on A
    new_B = fit_B_on_A(A,B,overlap_A,overlap_B)
    
    # remove overlapping selection from A used for fit
    new_A = remove_overlap(A,overlap_A)
    
    # splits topology in leading chain and remainder (not leading chain(s))
    A_active, A_passive = split_chain_topology(new_A,leading_chain)
    
    if dimer_b:
        # splits topology in leading chain and remainder (not adding chain(s))
        B_active, B_passive = split_chain_topology(new_B,adding_chain)
            
        # add B to active part of A (and make sure they are in same chain)
        if reverse:
            temp = merge_chain_topology(B_active,A_active,keep_resSeq=keep_resSeq)
        else:
            temp = merge_chain_topology(A_active,B_active,keep_resSeq=keep_resSeq)
            
        C_temp = temp.stack(A_passive,keep_resSeq=keep_resSeq)
        C =  C_temp.stack(B_passive,keep_resSeq=keep_resSeq)
    else:
        # add B to active part of A (and make sure they are in same chain)
        if reverse:
            temp = merge_chain_topology(new_B,A_active)
        else:
            temp = merge_chain_topology(A_active,new_B)
        # combine passive part with new structure (active part of A and B)
        C = temp.stack(A_passive,keep_resSeq=keep_resSeq)
        
    return C

def add_dimer(traj, chainid = 0,  site_map=None, verbose=False, segment='random'):
    
    s1_pairs = [['s1', 'h3'],['h3','s2'],['s2','l2'],['l2','dbd']]
    s2_pairs = [['s2','h3'],['h3','s1'],['s2', 'l2'],['l2','dbd']]
    

    for idx,pair in enumerate(s2_pairs):
        if idx > 0:
            leading_chain = 0
        else:
            leading_chain = chainid
        traj = add_pair(traj,pair,site_map,leading_chain=leading_chain,verbose=verbose,segment=segment)

    for idx,pair in enumerate(s1_pairs):
        if idx > 0:
            leading_chain = 0
        else:
            leading_chain =  chainid + 2
        traj = add_pair(traj,pair,site_map,leading_chain=leading_chain,verbose=verbose,segment=segment)
    return traj


# initialize number of dimers
traj = None
dimers = 2

i = 0
for idx in range(dimers):
    print(idx)
    traj = add_dimer(traj, chainid=i, site_map=site_map, segment='random')
    i+=2

# show first frame
print([c.n_residues for c in traj.top.chains])


view = nv.show_mdtraj(traj.atom_slice(traj.top.select(f'chainid 0 to {(dimers*2)-1}')))
view

In [22]:
# initialize number of dimers
traj = None
dimers = 2

i = 0
for idx in range(dimers):
    print(idx)
    traj = add_dimer(traj, chainid=i, site_map=site_map, segment='random')
    i+=2

# show first frame
print([c.n_residues for c in traj.top.chains])


view = nv.show_mdtraj(traj.atom_slice(traj.top.select(f'chainid 0 to {(dimers*2)-1}')))
view

0
1
[137, 137, 137, 137, 33, 33]


NGLWidget()

In [ ]:
# # Connections between sites
# pairs = [['s1','h3'],
#          ['h3','s2'],
#          ['s2','l2'],
#          ['l2','dbd']]

# # number of residues to overlap in the superimposition
# n = 2

# # define segments of the protein
# segments = {'s1':np.arange(0,41+n),
#             'h3':np.arange(41-n,53+n),
#             's2':np.arange(53-n,82+n),
#             'l2':np.arange(82-n,95+n),
#             'dbd':np.arange(95-n,137)}

# # get structures of the different sites
# k = 100 # inteval of frames to take
# s1 = get_site_structures(s1s1,segments,site='s1')[::k]
# s2 = get_site_structures(s2s2,segments,site='s2')[::k]

# h3_s1s1 = get_segment_structures(s1s1,segments,site='h3')
# h3_s2s2 = get_segment_structures(s2s2,segments,site='h3')
# h3 = md.join([h3_s1s1,h3_s2s2])[::k]

# l2_s1s1 = get_segment_structures(s1s1,segments,site='l2')
# l2_s2s2 = get_segment_structures(s2s2,segments,site='l2')
# l2 = md.join([l2_s1s1,l2_s2s2])[::k]

# dbd_s1s1 = get_segment_structures(s1s1,segments,site='dbd')
# dbd_s2s2 = get_segment_structures(s2s2,segments,site='dbd')
# dbd = md.join([dbd_s1s1,dbd_s2s2])[::k]

In [ ]:
    # Connections between sites
    pairs = [['s1','h3'],
            ['h3','s2'],
            ['s2','l2'],
            ['l2','dbd']]

In [1]:
class SiteMapper:
    def __init__(self, s1s1, s2s2, n=2, k=100):
        
        self.n = n # number of residues to overlap in the superimposition
        self.k = k # inteval of frames to take
        self.s1s1 = s1s1[::k]
        self.s2s2 = s2s2[::k]


    def get_site_map(self):

        # define segments of the protein
        segments = {'s1':np.arange(0,41+self.n),
                    'h3':np.arange(41-self.n,53+self.n),
                    's2':np.arange(53-self.n,82+self.n),
                    'l2':np.arange(82-self.n,95+self.n),
                    'dbd':np.arange(95-self.n,137)}

        # get structures of the different sites
        s1 = self.get_site_structures(self.s1s1,segments,site='s1')
        s2 = self.get_site_structures(self.s2s2,segments,site='s2')

        h3_s1s1 = self.get_segment_structures(self.s1s1,segments,site='h3')
        h3_s2s2 = self.get_segment_structures(self.s2s2,segments,site='h3')
        h3 = md.join([h3_s1s1,h3_s2s2])

        l2_s1s1 = self.get_segment_structures(self.s1s1,segments,site='l2')
        l2_s2s2 = self.get_segment_structures(self.s2s2,segments,site='l2')
        l2 = md.join([l2_s1s1,l2_s2s2])

        dbd_s1s1 = self.get_segment_structures(self.s1s1,segments,site='dbd')
        dbd_s2s2 = self.get_segment_structures(self.s2s2,segments,site='dbd')
        dbd = md.join([dbd_s1s1,dbd_s2s2])


        # map of the different sites
        site_map = {'s1':s1,
                'h3':h3,
                's2':s2,
                'l2':l2,
                'dbd':dbd}
        
        return site_map


    def check_selection(self,top,selection):
        if selection == 'CA':
            indices = top.select('name CA')
        elif selection == 'backbone':
            indices = top.select('backbone')
        elif selection == 'sidechain':
            indices = top.select('sidechain')
        else:
            indices = top.select('all')   
        return indices 
    
    def get_monomer_domain_indices(self,top,domain,chain=0,selection=None):
        residues = np.array(top._chains[chain]._residues)
        indices = self.check_selection(top,selection)
        return [at.index for res in residues[domain] for at in res.atoms if at.index in indices]

    def get_segment_structures(self,traj,segments,site='dbd'):
        chain_a = self.get_monomer_domain_indices(top=traj.top, domain=segments[site], chain=0, selection=None)
        chain_b = self.get_monomer_domain_indices(top=traj.top, domain=segments[site], chain=1, selection=None)
        A = traj.atom_slice(chain_a)
        B = traj.atom_slice(chain_b)
        return md.join([A,B])

    def get_site_structures(self, traj,segements,site='s1'):
        chain_a = self.get_monomer_domain_indices(top=traj.top, domain=segements[site], chain=0, selection=None)
        chain_b = self.get_monomer_domain_indices(top=traj.top, domain=segements[site], chain=1, selection=None)
        return traj.atom_slice(np.sort(chain_a+chain_b))

    def show_domain(self, system,domains, domain):
        # shows first frame
        top = system.top
        view = nv.show_mdtraj(system[0])
        view.clear()
        indices = self.get_monomer_domain_indices(top,domains[domain],chain=0)
        view.add_representation('cartoon',selection=[i for i in  top.select('all') if i not in indices],color='cornflowerblue')
        top = system.topology
        chain_id = 0
        indices = self.get_monomer_domain_indices(top,domains[domain],chain=chain_id)
        view.add_representation('cartoon',selection=indices,color='gold')
        top = system.topology
        chain_id = 1
        indices = self.get_monomer_domain_indices(top,domains[domain],chain=chain_id)
        view.add_representation('cartoon',selection=indices,color='red')
        return view

In [6]:
##### Functions to split and manage segements of protein structures #####

def check_selection(top,selection):
    if selection == 'CA':
        indices = top.select('name CA')
    elif selection == 'backbone':
        indices = top.select('backbone')
    elif selection == 'sidechain':
        indices = top.select('sidechain')
    else:
        indices = top.select('all')   
    return indices 

def get_monomer_domain_indices(top,domain,chain=0,selection=None):
    residues = np.array(top._chains[chain]._residues)
    indices = check_selection(top,selection)
    return [at.index for res in residues[domain] for at in res.atoms if at.index in indices]

def get_segment_structures(traj,segments,site='dbd'):
    chain_a = get_monomer_domain_indices(top=traj.top, domain=segments[site], chain=0, selection=None)
    chain_b = get_monomer_domain_indices(top=traj.top, domain=segments[site], chain=1, selection=None)
    A = traj.atom_slice(chain_a)
    B = traj.atom_slice(chain_b)
    return md.join([A,B])

def get_site_structures(traj,segements,site='s1'):
    chain_a = get_monomer_domain_indices(top=traj.top, domain=segements[site], chain=0, selection=None)
    chain_b = get_monomer_domain_indices(top=traj.top, domain=segements[site], chain=1, selection=None)
    return traj.atom_slice(np.sort(chain_a+chain_b))

def show_domain(system,domains, domain):
    # shows first frame
    top = system.top
    view = nv.show_mdtraj(system[0])
    view.clear()
    indices = get_monomer_domain_indices(top,domains[domain],chain=0)
    view.add_representation('cartoon',selection=[i for i in  top.select('all') if i not in indices],color='cornflowerblue')
    top = system.topology
    chain_id = 0
    indices = get_monomer_domain_indices(top,domains[domain],chain=chain_id)
    view.add_representation('cartoon',selection=indices,color='gold')
    top = system.topology
    chain_id = 1
    indices = get_monomer_domain_indices(top,domains[domain],chain=chain_id)
    view.add_representation('cartoon',selection=indices,color='red')
    return view

##### Functions to build protein filaments using segements and rotation, transation transformation fits #####

def get_rot_and_trans(subtraj_A,subtraj_B):
    
    """ fit only works now on a single frame (mdtraj returns xyz with shape (n_frames, atoms, xyz) 
         even for single frame trajs so hence the xyz[0]"""
    
    # load super imposer
    sup = SVDSuperimposer()

    # Set the coords, y will be rotated and translated on x
    x = subtraj_A.xyz[0]
    y = subtraj_B.xyz[0]
    sup.set(x, y)

    # Do the leastsquared fit
    sup.run()

    # Get the rms
    rms = sup.get_rms()

    # Get rotation (right multiplying!) and the translation
    rot, tran = sup.get_rotran()
    
    # now we have the instructions to rotate B on A
    return rot,tran,rms

def apply_superimposition(traj, rot, tran):
    
    # get xyz coordinates
    xyz = traj.xyz[0]
    
    # rotate subject on target
    new_xyz = dot(xyz, rot) + tran

    # replace coordinates of traj
    traj.xyz = new_xyz
    return traj

def fit_B_on_A(A, B, overlap_A, overlap_B):
    # create trajs containing only the selections
    subtraj_A = A.atom_slice(overlap_A)
    subtraj_B = B.atom_slice(overlap_B)

    # obtain instructions to rotate and translate B on A based on substraj structures
    rot, tran, rms = get_rot_and_trans(subtraj_A,subtraj_B)

    # do the superimposition of B on A and subsitute old with new xyz of B
    return apply_superimposition(B, rot, tran)
    


##### Functions to edit and mange topology and trajectory #####

def get_overlap_indices(top,n,chain=0,terminus=None):
    residues = np.array(top._chains[chain]._residues)
    if terminus == 'N_terminus': # get residues at end of chain
        s = residues[len(residues)-n*2:len(residues)]
        return [at.index for res in s for at in res.atoms]
    elif terminus == 'C_terminus': # get residues at beginning of chain
        s = residues[:n*2]
        return [at.index for res in s for at in res.atoms]
    else:
        print('No terminus')
        
def check_if_dimerization(site):
    if 's' in site:
        return True
    else:
        return False
    
def get_termini(site_x,site_y):
    chain_order = np.array(['s1','h3','s2','l2','dbd'])
    x = np.argwhere(chain_order==site_x)
    y = np.argwhere(chain_order==site_y)
    if x < y:
        return ['N_terminus','C_terminus']
    elif x > y:
        return ['C_terminus','N_terminus']

def check_overlaps(overlap_A,overlap_B):

    if len(overlap_A) != len(overlap_B):
        print(len(overlap_A),len(overlap_B))
        print('Something went wrong with finding the overlaps') 
    else:
        False

def remove_overlap(traj,overlap):
     return traj.atom_slice([at.index for at in traj.top.atoms if at.index not in overlap])
    
def split_chain_topology(traj,leading_chain):
    # split part of A in chain that is being extended and that is not
    traj_active = traj.atom_slice(traj.top.select(f'chainid {leading_chain}'))
    traj_passive = traj.atom_slice(traj.top.select(f'not chainid {leading_chain}'))
    return traj_active, traj_passive

def merge_chain_topology(A,B,keep_resSeq=True):
    C = A.stack(B,keep_resSeq=keep_resSeq)
    top = C.top
    # Merge two tops (with two chains or more) to a top of one chain 
    out = md.Topology()
    c = out.add_chain()
    for chain in top.chains:

        for residue in chain.residues:
            r = out.add_residue(residue.name, c, residue.resSeq, residue.segment_id)
            for atom in residue.atoms:
                out.add_atom(atom.name, atom.element, r, serial=atom.serial)
    #     for bond in top.bonds:
    #         a1, a2 = bond
    #         out.add_bond(a1, a2, type=bond.type, order=bond.order)
    out.create_standard_bonds() #rare manier om bonds te maken, maar werkt
    C.top = out 
    return C

##### Functions to orchestrate the addition of protein segments ##### 

def add_pair(traj,pair,site_map,leading_chain=0,adding_chain=0,verbose=False,reverse=False,segment='fixed'):

    keep_resSeq = False
    A,B,C=None,None,None
    site_a, site_b = pair
    if verbose:
        print(site_a,site_b)
    
    # get segment structures
    if not traj:
        if segment == 'fixed':
            x,y = 40,90 # get fixed frame for segment
        elif segment == 'random':
            k = len(site_map[site_a])
            l = len(site_map[site_b])
            x,y = np.random.randint(0,k,1)[0],np.random.randint(0,l,1)[0]
        A = site_map[site_a][x]
        B = site_map[site_b][y]
    else:
        if segment == 'fixed':
            z = 20
        elif segment == 'random':
            k = len(site_map[site_b])
            z = np.random.randint(0,k,1)[0]
        A = traj
        B = site_map[site_b][z]
        
    # check if site had dimerization site
    dimer_a = check_if_dimerization(site_a)
    dimer_b = check_if_dimerization(site_b)
    if verbose:
        print(dimer_a, dimer_b)
        
    # get_termini of site a and b
    terminus_a, terminus_b = get_termini(site_a,site_b)
    
    # determine growth direction (forward, or reverse)
    if terminus_a == 'C_terminus':
        reverse = True
    else:
        reverse = False
        
    # get atom indices of overlapping segements
    overlap_A = get_overlap_indices(A.top,n,chain=leading_chain,terminus=terminus_a)
    overlap_B = get_overlap_indices(B.top,n,chain=adding_chain,terminus=terminus_b)
    
    # make sure overlapping indices are consistent
    check = check_overlaps(overlap_A,overlap_B)
    if check:
        return check
    
    # obtain superimposition of B on A
    new_B = fit_B_on_A(A,B,overlap_A,overlap_B)
    
    # remove overlapping selection from A used for fit
    new_A = remove_overlap(A,overlap_A)
    
    # splits topology in leading chain and remainder (not leading chain(s))
    A_active, A_passive = split_chain_topology(new_A,leading_chain)
    
    if dimer_b:
        # splits topology in leading chain and remainder (not adding chain(s))
        B_active, B_passive = split_chain_topology(new_B,adding_chain)
            
        # add B to active part of A (and make sure they are in same chain)
        if reverse:
            temp = merge_chain_topology(B_active,A_active,keep_resSeq=keep_resSeq)
        else:
            temp = merge_chain_topology(A_active,B_active,keep_resSeq=keep_resSeq)
            
        C_temp = temp.stack(A_passive,keep_resSeq=keep_resSeq)
        C =  C_temp.stack(B_passive,keep_resSeq=keep_resSeq)
    else:
        # add B to active part of A (and make sure they are in same chain)
        if reverse:
            temp = merge_chain_topology(new_B,A_active)
        else:
            temp = merge_chain_topology(A_active,new_B)
        # combine passive part with new structure (active part of A and B)
        C = temp.stack(A_passive,keep_resSeq=keep_resSeq)
        
    return C

def add_dimer(traj, chainid = 0,  site_map=None, verbose=False, segment='random'):
    
    s1_pairs = [['s1', 'h3'],['h3','s2'],['s2','l2'],['l2','dbd']]
    s2_pairs = [['s2','h3'],['h3','s1'],['s2', 'l2'],['l2','dbd']]
    

    for idx,pair in enumerate(s2_pairs):
        if idx > 0:
            leading_chain = 0
        else:
            leading_chain = chainid
        traj = add_pair(traj,pair,site_map,leading_chain=leading_chain,verbose=verbose,segment=segment)

    for idx,pair in enumerate(s1_pairs):
        if idx > 0:
            leading_chain = 0
        else:
            leading_chain =  chainid + 2
        traj = add_pair(traj,pair,site_map,leading_chain=leading_chain,verbose=verbose,segment=segment)
    return traj

In [3]:
# Load H-NS s1s1 dimers
loc_dimers = '/Users/thor/surfdrive/Projects/pymdna/studies/1_protein-dna_filament/FilamentsDNA/tests/data/0_s1s1/drytrajs/'
short_trajs = [md.load(loc_dimers+f'dry_{i}.xtc',top=loc_dimers+f'dry_{i}.pdb').remove_solvent() for i in range(0,16)]
#start_open = md.load(loc_dimers+f'dry_open.xtc',top=loc_dimers+f'dry_open.pdb').remove_solvent()
#start_closed = md.load(loc_dimers+f'dry_closed.xtc',top=loc_dimers+f'dry_closed.pdb').remove_solvent()

#s1s1 = md.join([start_open,start_closed,md.join(short_trajs)])
s1s1 = md.join(short_trajs)

# Load H-NS s2s2 dimers
loc_dimers = '/Users/thor/surfdrive/Projects/pymdna/studies/1_protein-dna_filament/FilamentsDNA/tests/data/1_s2s2/drytrajs/'
short_trajs = [md.load(loc_dimers+f'dry_{i}.xtc',top=loc_dimers+f'dry_{i}.pdb').remove_solvent() for i in range(0,10)]

s2s2 = md.join(short_trajs)

In [7]:
# Connections between sites
pairs = [['s1','h3'],
         ['h3','s2'],
         ['s2','l2'],
         ['l2','dbd']]

# number of residues to overlap in the superimposition
n = 2

# define segments of the protein
segments = {'s1':np.arange(0,41+n),
            'h3':np.arange(41-n,53+n),
            's2':np.arange(53-n,82+n),
            'l2':np.arange(82-n,95+n),
            'dbd':np.arange(95-n,137)}

# get structures of the different sites
k = 100 # inteval of frames to take
s1 = get_site_structures(s1s1,segments,site='s1')[::k]
s2 = get_site_structures(s2s2,segments,site='s2')[::k]

h3_s1s1 = get_segment_structures(s1s1,segments,site='h3')
h3_s2s2 = get_segment_structures(s2s2,segments,site='h3')
h3 = md.join([h3_s1s1,h3_s2s2])[::k]

l2_s1s1 = get_segment_structures(s1s1,segments,site='l2')
l2_s2s2 = get_segment_structures(s2s2,segments,site='l2')
l2 = md.join([l2_s1s1,l2_s2s2])[::k]

dbd_s1s1 = get_segment_structures(s1s1,segments,site='dbd')
dbd_s2s2 = get_segment_structures(s2s2,segments,site='dbd')
dbd = md.join([dbd_s1s1,dbd_s2s2])[::k]

# map of the different sites
site_map = {'s1':s1,
           'h3':h3,
           's2':s2,
           'l2':l2,
           'dbd':dbd}

In [13]:
# initialize number of dimers
traj = None
dimers = 4

i = 0
for idx in range(dimers):
    print(idx)
    traj = add_dimer(traj, chainid=i, site_map=site_map, segment='random')
    i+=2

# show first frame
print([c.n_residues for c in traj.top.chains])

0
1
2
3
[137, 137, 137, 137, 137, 137, 137, 137, 33, 33]


In [14]:
view = nv.show_mdtraj(traj.atom_slice(traj.top.select(f'chainid 0 to {(dimers*2)-1}')))
view

NGLWidget()

In [31]:
import openmm.app as app
import openmm.unit as unit
import openmm as mm
from mdtraj.reporters import HDF5Reporter

time = 100 * unit.picoseconds
time_step = 2 * unit.femtoseconds
temperature = 310 * unit.kelvin
steps = int(time/time_step)
print(steps)

pdb = traj.atom_slice(traj.top.select(f'chainid 0 to {(dimers*2)-1}'))
topology = pdb.topology.to_openmm()
modeller = app.Modeller(topology, pdb.xyz[0])

50000


In [32]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
_ = modeller.addHydrogens(forcefield)

In [33]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.CutoffNonPeriodic)
integrator = mm.LangevinIntegrator(temperature, 1.0/unit.picoseconds, time_step)

simulation = app.Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.reporters.append(HDF5Reporter(f'./hns_{dimers}'+'.h5', 100))
simulation.reporters.append(app.StateDataReporter(f'./output_hns_{dimers}.csv', 100, step=True, potentialEnergy=True, temperature=True,speed=True))
simulation.minimizeEnergy()
simulation.step(steps)

In [34]:
simulation.reporters[0].close()
traj_md = md.load(f'./hns_{dimers}'+'.h5')

/Users/thor/opt/anaconda3/envs/mdna/lib/python3.11/site-packages/mdtraj/core/trajectory.py:439: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn('top= kwargs ignored since this file parser does not support it')


In [35]:
view = nv.show_mdtraj(traj_md)
# view.clear()
# view.add_representation('ball+stick',selection='not water')
view


NGLWidget(max_frame=499)